# Analysing Supervised Learning Models With Imandra

Two of the most common tasks within supervised learning (and machine learning more generally) are classification and regression. In this notebook we show how two of the most common kinds of model used to perform this task, neural networks and random forests, can be analysed using Imandra. For each task we use a common real-world benchmark dataset from the UCI Machine Learning Repository. We'll mostly be working with reals in this notebook so we'll start by installing a pretty printer so that we're not overrun with digits.

In [24]:
let pp_approx fmt r =
 CCFormat.fprintf fmt "%s" (Real.to_string_approx r) [@@program]

#install_printer pp_approx

val pp_approx : CCFormat.t -> Q.t -> unit = <fun>


## Classification

In a classification task we want to learn to predict the label of a datapoint based on previous data. In the classic [Wisconsin Breast Cancer (Diagnostic) dataset](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)) the task is to predict whether the cancer is benign or malignant based on the features of cell nuclei. In the dataset we have the following variables:

```
1. ID number
2. Diagnosis (malignant or benign)
3-32. Real values for the mean, standard error, and the 'worst' value for each cell nucleus's:
      a) Radius (mean of distances from center to points on the perimeter)
      b) Texture (standard deviation of gray-scale values)
      c) Perimeter
      d) Area
      e) Smoothness (local variation in radius lengths)
      f) Compactness (perimeter^2 / area - 1.0)
      g) Concavity (severity of concave portions of the contour)
      h) Concave points (number of concave portions of the contour)
      i) Symmetry 
      j) Fractal dimension ("coastline approximation" - 1)
```

As is standard practice we pre-process the data before learning.

We then split the data into training (80%) and test (20%) sets and use Scikit-Learn to learn a random forest of 5 decision trees. Each tree is then converted to IML using a short Python script and can be reasoned about using Imandra.

## Regression

In a regression task we want to learn to predict the value(s) of some variable(s) based on previous data. In the commonly used [Forest Fires dataset](https://archive.ics.uci.edu/ml/datasets/forest+fires) the aim is to predict the area burned by forest fires, in the northeast region of Portugal, by using meteorological and other data. This is a fairly difficult task and while the neural network below doesn't achieve state-of-the-art performance, it's enough to demonstrate how we can analyse relatively simple models in Imandra. In the dataset we have the following variables:

```
1. X-axis spatial coordinate within the Montesinho park map
2. Y-axis spatial coordinate within the Montesinho park map
3. Month of the year
4. Day of the week
5. FFMC index from the FWI system
6. DMC index from the FWI system
7. DC index from the FWI system
8. ISI index from the FWI system
9. Temperature in degrees Celsius
10. Relative humidity in %
11. Wind speed in km/h
12. Outside rain in mm/m2
13. The burned area of the forest in ha
```

We again pre-process the data before learning by transforming the month and day variables into a numerical value and applying a `sin` transformation (so similar times are close in value), removing outliers and applying an approximate log transformation to the area variable (as recommended in the dataset description), scaling each variable to lie between 0 and 1, and shuffling. We then split the data into training (80%) and test (20%) sets and use Keras to learn a simple feed-forward neural network with one (6 neuron) hidden layer, ReLU activation functions, and stochastic gradient descent to optimise the mean squared error. After saving our model as an `.h5` file we use a short script to extract the network into an IML file and reason about it using Imandra.

In [23]:
let relu x = Real.(if x > 0.0 then x else 0.0);;

let linear x = Real.(x)

let layer_0 (x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, x_10, x_11) = let open Real in
let y_0 = relu @@ (0.29736)*x_0 + (-0.03284)*x_1 + (0.33764)*x_2 + (0.31006)*x_3 + (0.38830)*x_4 + (-0.11433)*x_5 + (-0.22615)*x_6 + (0.30568)*x_7 + (0.24738)*x_8 + (-0.27070)*x_9 + (-0.32861)*x_10 + (-0.04558)*x_11 + -0.11147 in
let y_1 = relu @@ (0.46853)*x_0 + (0.00982)*x_1 + (0.07668)*x_2 + (0.13228)*x_3 + (-0.26862)*x_4 + (-0.24804)*x_5 + (-0.23180)*x_6 + (0.00997)*x_7 + (0.41881)*x_8 + (-0.13255)*x_9 + (0.23491)*x_10 + (0.01547)*x_11 + 0.02454 in
let y_2 = relu @@ (-0.12470)*x_0 + (0.21374)*x_1 + (-0.00985)*x_2 + (0.29477)*x_3 + (-0.16725)*x_4 + (0.49801)*x_5 + (0.18420)*x_6 + (0.27119)*x_7 + (0.03552)*x_8 + (-0.44214)*x_9 + (-0.21708)*x_10 + (-0.24895)*x_11 + -0.09070 in
let y_3 = relu @@ (-0.11102)*x_0 + (0.31046)*x_1 + (-0.17888)*x_2 + (-0.47995)*x_3 + (0.32406)*x_4 + (-0.43446)*x_5 + (0.12602)*x_6 + (-0.28886)*x_7 + (0.25140)*x_8 + (-0.58971)*x_9 + (-0.50674)*x_10 + (-0.10969)*x_11 + -0.17643 in
let y_4 = relu @@ (-0.07817)*x_0 + (0.22493)*x_1 + (-0.19861)*x_2 + (-0.53939)*x_3 + (-0.28887)*x_4 + (-0.26368)*x_5 + (-0.30336)*x_6 + (0.21727)*x_7 + (-0.27373)*x_8 + (-0.39930)*x_9 + (-0.56700)*x_10 + (-0.15258)*x_11 + 0.00000 in
let y_5 = relu @@ (-0.18358)*x_0 + (-0.21640)*x_1 + (0.05054)*x_2 + (0.11789)*x_3 + (-0.07178)*x_4 + (0.17706)*x_5 + (-0.32810)*x_6 + (-0.40519)*x_7 + (-0.39900)*x_8 + (0.43551)*x_9 + (-0.00760)*x_10 + (0.17371)*x_11 + -0.14378 in
(y_0, y_1, y_2, y_3, y_4, y_5);;

let layer_1 (x_0, x_1, x_2, x_3, x_4, x_5) = let open Real in
let y_0 = linear @@ (-0.45929)*x_0 + (0.53751)*x_1 + (0.50732)*x_2 + (-0.85146)*x_3 + (0.87902)*x_4 + (-0.57863)*x_5 + 0.33179 in
(y_0);;

let nn (x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, x_10, x_11) = let open Real in 
(x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, x_10, x_11)
|> layer_0
|> layer_1
;;

val relu : real -> real = <fun>
val linear : 'a -> 'a = <fun>
val layer_0 :
  real * real * real * real * real * real * real * real * real * real *
  real * real -> real * real * real * real * real * real = <fun>
val layer_1 : real * real * real * real * real * real -> real = <fun>
val nn :
  real * real * real * real * real * real * real * real * real * real *
  real * real -> real = <fun>


Given the description of the dataset above we can create some custom input types in Imandra for our model:

In [5]:
type month = Jan | Feb | Mar | Apr | May | Jun| Jul | Aug | Sep | Oct | Nov | Dec;;
type day = Mon | Tue | Wed | Thu | Fri | Sat | Sun;;
 
type input = {
  x : int;
  y : int;
  month : month;
  day : day;
  ffmc : real;
  dmc : real;
  dc : real;
  isi : real;
  temp : real;
  rh : real;
  wind : real;
  rain : real;
};;

type month =
    Jan
  | Feb
  | Mar
  | Apr
  | May
  | Jun
  | Jul
  | Aug
  | Sep
  | Oct
  | Nov
  | Dec
type day = Mon | Tue | Wed | Thu | Fri | Sat | Sun
type input = {
  x : int;
  y : int;
  month : month;
  day : day;
  ffmc : real;
  dmc : real;
  dc : real;
  isi : real;
  temp : real;
  rh : real;
  wind : real;
  rain : real;
}


However, remember that we pre-processed our data. To make things easier we'll add in a function applying this transform to each input variable. Equally, we'll need to convert back to hectares for our output variable. Here we simply use some minimum and maximum values extracted during our data pre-processing stage. After that we can define a full model which combines these pre/post-processing steps and the network above.

In [6]:
let month_2_num month = let open Real in
if month = Jan then 0.134 else
if month = Feb then 0.500 else
if month = Mar then 1.000 else
if month = Apr then 1.500 else
if month = May then 1.866 else
if month = Jun then 2.000 else
if month = Jul then 1.866 else
if month = Aug then 1.500 else
if month = Sep then 1.000 else
if month = Oct then 0.500 else
if month = Nov then 0.133 else
0.000;;

let day_2_num day = let open Real in
if day = Mon then 0.377 else
if day = Tue then 1.223 else
if day = Wed then 1.901 else
if day = Thu then 1.901 else
if day = Fri then 1.223 else
if day = Sat then 0.377 else
0.000;;

let process_input input = let open Real in
let real_month = month_2_num input.month in
let real_day = day_2_num input.day in
let real_x = Real.of_int(input.x) in
let real_y = Real.of_int(input.y) in
let x_0 =  (real_x     - 1.0)  / (9.0   - 1.0)  in
let x_1 =  (real_y     - 2.0)  / (9.0   - 2.0)  in
let x_2 =  (real_month - 0.0)  / (2.0   - 0.0)  in
let x_3 =  (real_day   - 0.0)  / (1.901 - 0.0)  in
let x_4 =  (input.ffmc - 18.7) / (96.2  - 18.7) in
let x_5 =  (input.dmc  - 1.1)  / (291.3 - 1.1)  in
let x_6 =  (input.dc   - 7.9)  / (860.6 - 7.9)  in
let x_7 =  (input.isi  - 0.0)  / (56.1  - 0.0)  in
let x_8 =  (input.temp - 2.2)  / (33.3  - 2.2)  in
let x_9 =  (input.rh   - 15.0) / (100.0 - 15.0) in
let x_10 =  (input.wind - 0.4) / (9.4   - 0.4)  in
let x_11 = (input.rain - 0.0)  / (6.40  - 0.0)  in
(x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, x_10, x_11);;

let process_output y_0 = let open Real in
let y = 4.44323 * y_0 in
if y <= 1.0 then (y - 0.00000) * 1.71828 else 
if y <= 2.0 then (y - 0.63212) * 4.67077 else 
if y <= 3.0 then (y - 1.49679) * 12.69648 else 
if y <= 4.0 then (y - 2.44700) * 34.51261 else 
(y - 3.42868) * 93.81501;;
(* if y <= 5.0 then (y - 3.42868) * 93.81501 else 
if y <= 6.0 then (y - 4.42194) * 255.01563 else 
(y - 5.41946) * 693.20436 ;; *)

let model input = input
|> process_input
|> nn
|> process_output;;

val month_2_num : month -> Q.t = <fun>
val day_2_num : day -> Q.t = <fun>
val process_input :
  input ->
  real * real * real * real * real * real * real * real * real * real *
  real * real = <fun>
val process_output : real -> real = <fun>
val model : input -> real = <fun>


As our model is fully executable we can both query it as well as find counterexamples, prove properties, apply logical side-conditions to the input, decompose its regions, and more. As a quick sanity check to make sure everything is working, let's run a datum from our dataset through the model. In particular, we'll input  `x = (8, 6, Aug, Sat, 93.7, 231.1, 715.1, 8.4, 26.9, 31.0, 3.6, 0.0)` which has an area of `y = 4.96` hectares in the data.

In [7]:
let x = {
  x = 8;
  y = 6;
  month = Aug;
  day = Sat;
  ffmc = 93.7;
  dmc = 231.1;
  dc = 715.1;
  isi = 8.4;
  temp = 26.9;
  rh = 31.0;
  wind = 3.6;
  rain = 0.0;};;
  
let y = model x;;

val x : input =
  {x = 8; y = 6; month = Aug; day = Sat; ffmc = 93.7; dmc = 231.1;
   dc = 715.1; isi = 8.4; temp = 26.9; rh = 31.; wind = 3.6; rain = 0.}
val y : real = 3.85719795238


Our answer is both similar to the recorded datapoint value and also to the value we get from our original Keras model, `3.85708846888`. The small disparity here is due to our rounding the weight values in our network to 5 decimal places when we extracted them to IML, though it wasn't necessary to do so. Now we'll use Imandra to generate an example for us with some particular side conditions.

In [14]:
instance (fun x -> model x >. 50.0 && x.temp = 20.0 && x.month = May);;

- : input -> bool = <fun>
module CX : sig val x : input end


Instance (after 0 steps, 0.035s):
 let (x : input) =
 {x = 6906; y = 0; month = May; day = Tue;
 ffmc =
 (Real.mk_of_string "3162864843347023023839315719933333075185411838151575706490341/149087588432410978823067913237692148861149669638118177000");
 dmc =
 (Real.mk_of_string "-11089606902904069462230798389413774114483562363213150294245693019/115542881035118508587877632759211415367390993969541587175000");
 dc = 0.;
 isi =
 (Real.mk_of_string "-119728537146976209957348319869340919662786540926468998020859/2709984075314722501826569864884403212482198000974331250");
 temp = 20.;
 rh =
 (Real.mk_of_string "3067371965318130600420405350674805342083901661935718822488071/19418971602540925813088677774657380734015293104124636500");
 wind =
 (Real.mk_of_string "-7653014215080570832206591258959167786880016558635278271889779/2567619578558189079730614061315809230386466532656479715000");
 rain =
 (Real.mk_of_string "-22653814448332663363912440142145928404099352168382753234865984/687755244256657789213557337852448900996374964104414209375")}

Instance call graph proof digraph "call graph" {
goal [label="(not\l (\<=.\l (if \<=.\l (+.\l (+.\l (+.\l (+.\l (+.\l (*. -20407311067/10000000000\l (if \<=.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+. (*. 3717/100000 (Real.of_int x.x))\l (*. -821/175000 (Real.of_int x.y)))\l (*. 8441/50000\l (if x.month = Jan then 67/500\l else\l if x.month = Feb then 1/2\l else\l if x.month = Mar then 1\l else if x.month = Apr then 3/2 else …)))\l (*. 15503/95050\l (if x.day = Mon then 377/1000\l else\l if x.day = Tue then 1223/1000\l else\l if x.day = Wed then 1901/1000\l else if x.day = Thu then 1901/1000 else …)))\l (*. 3883/775000 x.ffmc))\l (*. -11433/29020000 x.dmc))\l (*. -4523/17054000 x.dc))\l (*. 3821/701250 x.isi))\l (*. 12369/1555000 x.temp))\l (*. -2707/850000 x.rh))\l (*. -32861/900000 x.wind))\l (*. -2279/320000 x.rain))\l 23704241183297452571/127754212462947000000\l then 0\l else\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+. -23704241183297452571/127754212462947000000\l (*. 3717/100000 (Real.of_int x.x)))\l (*. -821/175000 (Real.of_int x.y)))\l (*. 8441/50000\l (if x.month = Jan then 67/500\l else\l if x.month = Feb then 1/2\l else\l if x.month = Mar then 1\l else if x.month = Apr then 3/2 else …)))\l (*. 15503/95050\l (if x.day = Mon then 377/1000\l else\l if x.day = Tue then 1223/1000\l else\l if x.day = Wed then 1901/1000\l else if x.day = Thu then 1901/1000 else …)))\l (*. 3883/775000 x.ffmc))\l (*. -11433/29020000 x.dmc))\l (*. -4523/17054000 x.dc))\l (*. 3821/701250 x.isi))\l (*. 12369/1555000 x.temp))\l (*. -2707/850000 x.rh))\l (*. -32861/900000 x.wind))\l (*. -2279/320000 x.rain))))\l (*. 23882805573/10000000000\l (if \<=.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+. (*. 46853/800000 (Real.of_int x.x))\l (*. 491/350000 (Real.of_int x.y)))\l (*. 1917/50000\l (if x.month = Jan then 67/500 else …)))\l …)\l …)\l …)\l …)\l …)\l …)\l …)\l …)\l …)\l -36781692621283221809/2555084249258940000000\l then 0 else …)))\l …)\l …)\l …)\l …)\l -4742192817/10000000000\l then … else …)\l 50)\l && …)\l&& …",shape=box,style=filled,color="cyan",fontname="courier",fontsize=14];
}
 Load graph Loading.. proof attempt summary full graph ground_instances 0 definitions 0 inductions 0 search_time 0.036s details Expand smt_stats num checks 1 arith assert lower 46 arith pivots 20 rlimit count 21728 mk clause 81 datatype occurs check 7 mk bool var 150 arith assert upper 44 datatype splits 4 decisions 72 arith add rows 151 arith bound prop 3 propagations 94 conflicts 9 arith fixed eqs 10 datatype accessor ax 10 arith conflicts 2 arith assert diseq 4 datatype constructor ax 7 num allocs 1362230817 final checks 1 added eqs 92 del clause 23 arith eq adapter 33 memory 19.270000 max memory 27.850000 Expand start[0.036s]
 >.
 (if <=.
 (*. 444323/100000
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (*. -45929/100000
 (if >.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+. (+. (*. 3717/12500 …) (*. -821/25000 …))
 (*. 8441/25000 …))
 (*. 15503/50000 …))
 (*. 3883/10000 …))
 (*. -11433/100000 …))
 (*. -4523/20000 …))
 (*. 3821/12500 …))
 (*. 12369/50000 …))
 (*. -2707/10000 …))
 (*. -32861/100000 …))
 (*. -2279/50000 …))
 -11147/100000)
 0
 then
 +.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+. (+. (*. 3717/12500 …) (*. -821/25000 …))
 (*. 8441/25000 …))
 (*. 15503/50000 …))
 (*. 3883/10000 …))
 (*. -11433/100000 …))
 (*. -4523/20000 …))
 (*. 3821/12500 …))
 (*. 12369/50000 …))
 (*. -2707/10000 …))
 (*. -32861/100000 …))
 (*. -2279/50000 …))
 -11147/100000
 else 0))
 (*. 53751/100000
 (if >.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+. (+. (*. 46853/100000 …) (*. 491/50000 …))
 (*. 1917/25000 …))
 (*. 3307/25000 …))
 (*. -13431/50000 …))
 (*. -6201/25000 …))
 (*. -1159/5000 …))
 (*. 997/100000 …))
 (*. 41881/100000 …))
 (*. -2651/20000 …))
 (*. 23491/100000 …))
 (*. 1547/100000 …))
 1227/50000)
 0
 then
 +.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+. (+. (*. 46853/100000 …) (*. 4

In [15]:
CX.x;;

- : input =
{x = 6906; y = 0; month = May; day = Tue; ffmc = 21214.8098752;
 dmc = -95978.2792636; dc = 0.; isi = -44180.5316266; temp = 20.;
 rh = 157957.487559; wind = -2980.58726417; rain = -32938.7738407}


This looks a bit funny however; notice how the unspecified input variables are unbounded. In general we might only care about the performance of our model when some reasonable bounds are placed on the input. For example, we can't have a negative windspeed, or a percentage humidity higher than 100. Using the description of each variable in the data (plus some reasonable assumptions about Portugal's climate) we can form the following condition describing valid inputs to the network.

In [16]:
let is_valid input = if 
    0 <= input.x && input.x <= 9 &&
    0 <= input.y && input.y <= 9 &&
    0.0 <=. input.ffmc && input.ffmc <=. 99.0 &&
    0.0 <=. input.dmc && input.dmc <=. 500.0 &&
    0.0 <=. input.dc && input.dc <=. 1000.0 &&
    0.0 <=. input.isi && input.isi <=. 100.0 &&
    0.0 <=. input.temp && input.temp <=. 40.0 &&
    0.0 <=. input.rh && input.rh <=. 100.0 &&
    0.0 <=. input.wind && input.wind <=. 15.0 &&
    0.0 <=. input.rain && input.rain <=. 15.0
    then true else false;;
    
instance (fun x -> model x >. 50.0 && x.temp = 20.0 && x.month = May && is_valid x)

val is_valid : input -> bool = <fun>
- : input -> bool = <fun>
module CX : sig val x : input end


Instance (after 0 steps, 0.826s):
 let (x : input) =
 {x = 9; y = 9; month = May; day = Sun;
 ffmc =
 (Real.mk_of_string "275092922514325882409935751081695706438241/620211691440996990094080200000000000000000");
 dmc =
 (Real.mk_of_string "15030912988749292730285792691971/270162300406486335063232500000");
 dc =
 (Real.mk_of_string "25664135277733508548217226823409/4486796812421498383050000000000");
 isi = 100.; temp = 20.; rh = 0.;
 wind =
 (Real.mk_of_string "52937189055433497168841872857760798237/2407604458251876112925113550000000000000");
 rain =
 (Real.mk_of_string "38832585318979928853256935306180256631759/2624583592422060730820923007812500000000000")}

Instance call graph proof digraph "call graph" {
goal [label="(((((((((((((((((((((not\l (\<=.\l (if \<=.\l (+.\l (+.\l (+.\l (+.\l (+.\l (*. -20407311067/10000000000\l (if \<=.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (*. 3717/100000\l (Real.of_int x.x))\l (*. -821/175000\l (Real.of_int x.y)))\l (*. 8441/50000\l (if x.month = Jan then 67/500\l else\l if x.month = Feb then 1/2\l else\l if x.month = Mar then 1\l else\l if x.month = Apr then 3/2\l else …)))\l (*. 15503/95050\l (if x.day = Mon then 377/1000\l else\l if x.day = Tue then 1223/1000\l else\l if x.day = Wed then 1901/1000\l else\l if x.day = Thu then 1901/1000\l else …)))\l (*. 3883/775000 x.ffmc))\l (*. -11433/29020000 x.dmc))\l (*. -4523/17054000 x.dc))\l (*. 3821/701250 x.isi))\l (*. 12369/1555000 x.temp))\l (*. -2707/850000 x.rh))\l (*. -32861/900000 x.wind))\l (*. -2279/320000 x.rain))\l 23704241183297452571/127754212462947000000\l then 0\l else\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l -23704241183297452571/127754212462947000000\l (*. 3717/100000\l (Real.of_int x.x)))\l (*. -821/175000\l (Real.of_int x.y)))\l (*. 8441/50000\l (if x.month = Jan then 67/500\l else\l if x.month = Feb then 1/2\l else\l if x.month = Mar then 1\l else\l if x.month = Apr then 3/2\l else …)))\l (*. 15503/95050\l (if x.day = Mon then 377/1000\l else\l if x.day = Tue then 1223/1000\l else\l if x.day = Wed then 1901/1000\l else\l if x.day = Thu then 1901/1000\l else …)))\l (*. 3883/775000 x.ffmc))\l (*. -11433/29020000 x.dmc))\l (*. -4523/17054000 x.dc))\l (*. 3821/701250 x.isi))\l (*. 12369/1555000 x.temp))\l (*. -2707/850000 x.rh))\l (*. -32861/900000 x.wind))\l (*. -2279/320000 x.rain))))\l (*. 23882805573/10000000000\l (if \<=.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (*. 46853/800000\l (Real.of_int x.x))\l (*. 491/350000\l (Real.of_int x.y)))\l (*. 1917/50000\l (if x.month = Jan then 67/500\l else …)))\l …)\l …)\l …)\l …)\l …)\l …)\l …)\l …)\l …)\l -36781692621283221809/2555084249258940000000\l then 0 else …)))\l …)\l …)\l …)\l …)\l -4742192817/10000000000\l then … else …)\l 50)\l && …)\l && …)\l && …)\l && …)\l && …)\l && …)\l && …)\l && …)\l && …)\l && …)\l && …)\l && …)\l && …)\l && …)\l && …)\l && …)\l && …)\l && …)\l && …)\l && …)\l && …)\l&& …",shape=box,style=filled,color="cyan",fontname="courier",fontsize=14];
}
 Load graph Loading.. proof attempt summary full graph ground_instances 0 definitions 0 inductions 0 search_time 0.826s details Expand smt_stats arith offset eqs 10 num checks 1 arith assert lower 204 arith pivots 450 rlimit count 1109042 mk clause 256 datatype occurs check 9 mk bool var 445 arith assert upper 227 datatype splits 10 decisions 240 arith add rows 5618 arith bound prop 17 propagations 885 interface eqs 1 conflicts 75 arith fixed eqs 70 datatype accessor ax 11 minimized lits 50 arith conflicts 32 arith assert diseq 52 datatype constructor ax 19 num allocs 1456477920 final checks 2 added eqs 574 del clause 114 arith eq adapter 185 memory 23.050000 max memory 27.850000 Expand start[0.826s]
 >.
 (if <=.
 (*. 444323/100000
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (*. -45929/100000
 (if >.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+. (+. (*. 3717/12500 …) (*. -821/25000 …))
 (*. 8441/25000 …))
 (*. 15503/50000 …))
 (*. 3883/10000 …))
 (*. -11433/100000 …))
 (*. -4523/20000 …))
 (*. 3821/12500 …))
 (*. 12369/50000 …))
 (*. -2707/10000 …))
 (*. -32861/100000 …))
 (*. -2279/50000 …))
 -11147/100000)
 0
 then
 +.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+. (+. (*. 3717/12500 …) (*. -821/25000 …))
 (*. 8441/25000 …))
 (*. 15503/50000 …))
 (*. 3883/10000 …))
 (*. -11433/100000 …))
 (*. -4523/20000 …))
 (*. 3821/12500 …))
 (*. 12369/50000 …))
 (*. -2707/10000 …))
 (*. -32861/100000 …))
 (*. -2279/50000 …))
 -11147/100000
 else 0))
 (*. 53751/100000
 (if >.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+. (+. (*. 46853/100000 …) (*. 491/50000 …))
 (*. 1917/25000 …))
 (*. 3

In [17]:
CX.x;;

- : input =
{x = 9; y = 9; month = May; day = Sun; ffmc = 0.443546818466;
 dmc = 55.6366042417; dc = 5.7199236673; isi = 100.; temp = 20.; rh = 0.;
 wind = 0.0219874941974; rain = 0.0147957129013}


These constraints mean it is slightly harder for Imandra to find a particular instance satisfying our original demand, but nonetheless it's possible. Now let's try something a bit more interesting. First of all let's check for one desirable property of the model, namely that it never outputs a negative area as a prediction.

In [18]:
verify (fun x ->  is_valid x ==> model x >=. 0.0)

- : input -> bool = <fun>
module CX : sig val x : input end


Counterexample (after 0 steps, 0.093s):
 let (x : input) =
 {x = 2; y = 6; month = Jan; day = Mon;
 ffmc =
 (Real.mk_of_string "44010981223089061143426314187553379825506171228246423359/462162458770373795508784983063879014865722656250000000");
 dmc = 0.;
 dc =
 (Real.mk_of_string "3660026789178049370319923028761249931194261383983686117939/8087030000272955996641351869169132397619646444612109375");
 isi = 100.;
 temp =
 (Real.mk_of_string "5347348073676834437421831798808223067484120091550718090478761/1814787604520177254734759355145795484731829818006352343750000");
 rh =
 (Real.mk_of_string "1755431830510768269675978440002442063680126801444362329/423260225747091206104462493366568414190276054687500000");
 wind =
 (Real.mk_of_string "118470842299852718280704817124273677781411472432156881359/345769443096334655589021650100640701463646789550781250000");
 rain =
 (Real.mk_of_string "590777127521058160504108491058244322202765766654400118603542230534073686128810596898135304434366521/42184382657389333078842671052115237509089113839456591419068056681519830308463493082618713378906250")}

Refuted call graph proof digraph "call graph" {
goal [label="not\l(not\l (((((((((((((((((((0 \<= x.x && x.x \<= 9) && 0 \<= x.y) && x.y \<= 9)\l && \<=. 0 x.ffmc)\l && \<=. x.ffmc 99)\l && \<=. 0 x.dmc)\l && \<=. x.dmc 500)\l && \<=. 0 x.dc)\l && \<=. x.dc 1000)\l && \<=. 0 x.isi)\l && \<=. x.isi 100)\l && \<=. 0 x.temp)\l && \<=. x.temp 40)\l && \<=. 0 x.rh)\l && \<=. x.rh 100)\l && \<=. 0 x.wind)\l && \<=. x.wind 15)\l && \<=. 0 x.rain)\l && \<=. x.rain 15)\l \|\| \>=.\l (if \<=.\l (+.\l (+.\l (+.\l (+.\l (+.\l (*. -20407311067/10000000000\l (if \<=.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+. (*. 3717/100000 (Real.of_int x.x))\l (*. -821/175000 (Real.of_int x.y)))\l (*. 8441/50000\l (if x.month = Jan then 67/500\l else\l if x.month = Feb then 1/2\l else\l if x.month = Mar then 1\l else if x.month = Apr then 3/2 else …)))\l (*. 15503/95050\l (if x.day = Mon then 377/1000\l else\l if x.day = Tue then 1223/1000\l else\l if x.day = Wed then 1901/1000\l else if x.day = Thu then 1901/1000 else …)))\l (*. 3883/775000 x.ffmc))\l (*. -11433/29020000 x.dmc))\l (*. -4523/17054000 x.dc))\l (*. 3821/701250 x.isi))\l (*. 12369/1555000 x.temp))\l (*. -2707/850000 x.rh))\l (*. -32861/900000 x.wind))\l (*. -2279/320000 x.rain))\l 23704241183297452571/127754212462947000000\l then 0\l else\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+. -23704241183297452571/127754212462947000000\l (*. 3717/100000 (Real.of_int x.x)))\l (*. -821/175000 (Real.of_int x.y)))\l (*. 8441/50000\l (if x.month = Jan then 67/500\l else\l if x.month = Feb then 1/2\l else\l if x.month = Mar then 1\l else if x.month = Apr then 3/2 else …)))\l (*. 15503/95050\l (if x.day = Mon then 377/1000\l else\l if x.day = Tue then 1223/1000\l else\l if x.day = Wed then 1901/1000\l else if x.day = Thu then 1901/1000 else …)))\l (*. 3883/775000 x.ffmc))\l (*. -11433/29020000 x.dmc))\l (*. -4523/17054000 x.dc))\l (*. 3821/701250 x.isi))\l (*. 12369/1555000 x.temp))\l (*. -2707/850000 x.rh))\l (*. -32861/900000 x.wind))\l (*. -2279/320000 x.rain))))\l (*. 23882805573/10000000000\l (if \<=.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+. (*. 46853/800000 (Real.of_int x.x))\l (*. 491/350000 (Real.of_int x.y)))\l (*. 1917/50000\l (if x.month = Jan then 67/500 else …)))\l …)\l …)\l …)\l …)\l …)\l …)\l …)\l …)\l …)\l -36781692621283221809/2555084249258940000000\l then 0 else …)))\l …)\l …)\l …)\l …)\l -4742192817/10000000000\l then … else …)\l 0)",shape=box,style=filled,color="cyan",fontname="courier",fontsize=14];
}
 Load graph Loading.. proof attempt summary full graph ground_instances 0 definitions 0 inductions 0 search_time 0.094s details Expand smt_stats num checks 1 arith assert lower 30 arith pivots 20 rlimit count 42521 mk clause 75 datatype occurs check 48 mk bool var 160 arith gcd tests 8 arith assert upper 27 datatype splits 4 decisions 101 arith add rows 183 propagations 135 arith patches 3 conflicts 6 arith fixed eqs 2 datatype accessor ax 18 arith conflicts 1 arith assert diseq 1 datatype constructor ax 7 num allocs 1643371239 final checks 4 added eqs 48 del clause 3 arith ineq splits 2 arith eq adapter 14 memory 17.680000 max memory 27.850000 Expand start[0.094s]
 0 <= :var_0:.x
 && :var_0:.x <= 9
 && 0 <= :var_0:.y
 && :var_0:.y <= 9
 && <=. 0 :var_0:.ffmc
 && <=. :var_0:.ffmc 99
 && <=. 0 :var_0:.dmc
 && <=. :var_0:.dmc 500
 && <=. 0 :var_0:.dc
 && <=. :var_0:.dc 1000
 && <=. 0 :var_0:.isi
 && <=. :var_0:.isi 100
 && <=. 0 :var_0:.temp
 && <=. :var_0:.temp 40
 && <=. 0 :var_0:.rh
 && <=. :var_0:.rh 100
 && <=. 0 :var_0:.wind
 && <=. :var_0:.wind 15
 && <=. 0 :var_0:.rain
 && <=. :var_0:.rain
 15
 ==> >=.
 (if <=.
 (*. 444323/100000
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (*. -45929/100000
 (if >.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+. (*. 3717/12500 …) (*. -821/25000 …))
 (*. 8441/25000 …))
 (*. 15503/50000 …))
 (*. 3883/10000 …))
 (*. -11433/100000 …))
 (*. -4523/20000 …))
 (*. 3821/12500 …))
 (*. 12

So it seems our model might be a bit noisy in terms of its predictions. As counterexamples are first class objects we can run them back through the model and check the output.

In [19]:
model CX.x

- : real = -0.07825771616


This value is indeed negative. Perhaps we're being too demanding though, let's try the same thing again but with a greater tolerance.

In [21]:
verify (fun x -> is_valid x ==> model x >=. (-5.0));;

- : input -> bool = <fun>


Proved proof call graph proof summary full graph ground_instances 0 definitions 0 inductions 0 search_time 2.235s details Expand smt_stats arith offset eqs 8 num checks 1 arith assert lower 422 arith pivots 1030 rlimit count 2868726 mk clause 610 mk bool var 588 restarts 1 arith assert upper 332 datatype splits 49 decisions 507 arith add rows 11109 arith bound prop 24 propagations 1951 conflicts 117 arith fixed eqs 89 datatype accessor ax 18 minimized lits 90 arith conflicts 58 arith assert diseq 112 datatype constructor ax 9 num allocs 2046984061 added eqs 671 del clause 386 arith eq adapter 262 memory 18.560000 max memory 27.850000 Expand start[2.235s]
 0 <= :var_0:.x
 && :var_0:.x <= 9
 && 0 <= :var_0:.y
 && :var_0:.y <= 9
 && <=. 0 :var_0:.ffmc
 && <=. :var_0:.ffmc 99
 && <=. 0 :var_0:.dmc
 && <=. :var_0:.dmc 500
 && <=. 0 :var_0:.dc
 && <=. :var_0:.dc 1000
 && <=. 0 :var_0:.isi
 && <=. :var_0:.isi 100
 && <=. 0 :var_0:.temp
 && <=. :var_0:.temp 40
 && <=. 0 :var_0:.rh
 && <=. :var_0:.rh 100
 && <=. 0 :var_0:.wind
 && <=. :var_0:.wind 15
 && <=. 0 :var_0:.rain
 && <=. :var_0:.rain
 15
 ==> >=.
 (if <=.
 (*. 444323/100000
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (*. -45929/100000
 (if >.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+. (*. 3717/12500 …) (*. -821/25000 …))
 (*. 8441/25000 …))
 (*. 15503/50000 …))
 (*. 3883/10000 …))
 (*. -11433/100000 …))
 (*. -4523/20000 …))
 (*. 3821/12500 …))
 (*. 12369/50000 …))
 (*. -2707/10000 …))
 (*. -32861/100000 …))
 (*. -2279/50000 …))
 -11147/100000)
 0
 then
 +.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+. (*. 3717/12500 …) (*. -821/25000 …))
 (*. 8441/25000 …))
 (*. 15503/50000 …))
 (*. 3883/10000 …))
 (*. -11433/100000 …))
 (*. -4523/20000 …))
 (*. 3821/12500 …))
 (*. 12369/50000 …))
 (*. -2707/10000 …))
 (*. -32861/100000 …))
 (*. -2279/50000 …))
 -11147/100000
 else 0))
 (*. 53751/100000
 (if >.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+. (*. 46853/100000 …) (*. 491/50000 …))
 (*. 1917/25000 …))
 (*. 3307/25000 …))
 (*. -13431/50000 …))
 (*. -6201/25000 …))
 (*. -1159/5000 …))
 (*. 997/100000 …))
 (*. 41881/100000 …))
 (*. -2651/20000 …))
 (*. 23491/100000 …))
 (*. 1547/100000 …))
 1227/50000)
 0
 then
 +.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+. (*. 46853/100000 …) (*. 491/50000 …))
 (*. 1917/25000 …))
 (*. 3307/25000 …))
 (*. -13431/50000 …))
 (*. -6201/25000 …))
 (*. -1159/5000 …))
 (*. 997/100000 …))
 (*. 41881/100000 …))
 (*. -2651/20000 …))
 (*. 23491/100000 …))
 (*. 1547/100000 …))
 1227/50000
 else 0)))
 (*. 12683/25000
 (if >.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+. (*. -1247/10000 …) (*. 10687/50000 …))
 (*. -197/20000 …))
 (*. 29477/100000 …))
 (*. -669/4000 …))
 (*. 49801/100000 …))
 (*. 921/5000 …))
 (*. 27119/100000 …))
 (*. 111/3125 …))
 (*. -22107/50000 …))
 (*. -5427/25000 …))
 (*. -4979/20000 …))
 -907/10000)
 0
 then
 +.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+. (*. -1247/10000 …) (*. 10687/50000 …))
 (*. -197/20000 …))
 (*. 29477/100000 …))
 (*. -669/4000 …))
 (*. 49801/100000 …))
 (*. 921/5000 …))
 (*. 27119/100000 …))
 (*. 111/3125 …))
 (*. -22107/50000 …))
 (*. -5427/25000 …))
 (*. -4979/20000 …))
 -907/10000
 else 0)))
 (*. -42573/50000
 (if >.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+. (*. -5551/50000 …) (*. 15523/50000 …))
 (*. -559/3125 …))
 (*. -9599/20000 …))
 (*. 16203/50000 …))
 (*. -21723/50000 …))
 (*. 6301/50000 …))
 (*. -14443/50000 …))
 (*. 1257/5000 …))
 (*. -58971/100000 …))
 (*. -25337/50000 …))
 (*. -10969/100000 …))
 -17643/100000)
 0
 then
 +.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+. (*. -5551/50000 …) (*. 15523/50000 …))
 (*. -559/3125 …))
 (*. -9599/20000 …))
 (*. 16203/50000 …))
 (*. -21723/50000 …))
 (*. 6301/50000 …))
 (*. -14443/50000 …))
 (*. 1257/5000 …))
 (*. -58971/100000 …))
 (*. -25337/50000 …))
 (*. -10969/100000 …))
 -17643/100000
 else 0)))
 (*. 43951/50000
 (if >.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 

Now let's test a hypothesis. All other things remaining equal, we would assume that the higher the temperature and the less rain, the larger the area that would be burned. Due to the imperfections of our model (because of limited data, stochasticity in training, the complicated patterns present in natural physical phenomena, and so on) this assertion is in fact easily falsifiable by Imandra. Let's restrict our setting in a sensible way to see if we can prove something slightly weaker:

* There is very little data from winter months, and so the model is unlikely to generalise well here
* We'll increase the tolerance in temperature to 10 degrees celsius
* We'll increase the tolerance in rainfall to 7.5 mm/m2
* We'll increase the tolerance in area burned to 20 hectares

In [22]:
let winter month = month = Oct || month = Nov || month = Dec || month = Jan || month = Feb;;

verify (fun x x1 x2 -> 
is_valid x1 &&
is_valid x2 &&
x1.x = x2.x &&
x1.y = x2.y &&
x1.month = x2.month &&
not (winter x1.month) &&
x1.day = x2.day &&
x1.ffmc = x2.ffmc &&
x1.dmc = x2.dmc &&
x1.dc = x2.dc &&
x1.isi = x2.isi &&
x1.rh = x2.rh &&
x1.wind = x2.wind &&
(x1.rain +. 10.0) <=. x2.rain &&
(x1.temp -. 10.0) >=. x2.temp ==>
(model x1 +. 20.0) >=. model x2);;

val winter : month -> bool = <fun>
- : 'a -> input -> input -> bool = <fun>


Unknown (Verification aborted by the solver (after 0 steps)) instances call graph proof Expand expanded blocked digraph "call graph" {
goal [label="not\l(not\l ((((((((((((((((((((((((((((((((((((((((((((((((((((0 \<= x1.x && x1.x \<= 9)\l && 0 \<= x1.y)\l && x1.y \<= 9)\l && \<=. 0 x1.ffmc)\l && \<=. x1.ffmc 99)\l && \<=. 0 x1.dmc)\l && \<=. x1.dmc 500)\l && \<=. 0 x1.dc)\l && \<=. x1.dc 1000)\l && \<=. 0 x1.isi)\l && \<=. x1.isi 100)\l && \<=. 0 x1.temp)\l && \<=. x1.temp 40)\l && \<=. 0 x1.rh)\l && \<=. x1.rh 100)\l && \<=. 0 x1.wind)\l && \<=. x1.wind 15)\l && \<=. 0 x1.rain)\l && \<=. x1.rain 15)\l && 0 \<= x2.x)\l && x2.x \<= 9)\l && 0 \<= x2.y)\l && x2.y \<= 9)\l && \<=. 0 x2.ffmc)\l && \<=. x2.ffmc 99)\l && \<=. 0 x2.dmc)\l && \<=. x2.dmc 500)\l && \<=. 0 x2.dc)\l && \<=. x2.dc 1000)\l && \<=. 0 x2.isi)\l && \<=. x2.isi 100)\l && \<=. 0 x2.temp)\l && \<=. x2.temp 40)\l && \<=. 0 x2.rh)\l && \<=. x2.rh 100)\l && \<=. 0 x2.wind)\l && \<=. x2.wind 15)\l && \<=. 0 x2.rain)\l && \<=. x2.rain 15)\l && x1.x = x2.x)\l && x1.y = x2.y)\l && x1.month = x2.month)\l && not\l ((((x1.month = Oct \|\| x1.month = Nov) \|\| x1.month = Dec)\l \|\| x1.month = Jan)\l \|\| x1.month = Feb))\l && x1.day = x2.day)\l && x1.ffmc = x2.ffmc)\l && x1.dmc = x2.dmc)\l && x1.dc = x2.dc)\l && x1.isi = x2.isi)\l && x1.rh = x2.rh)\l && x1.wind = x2.wind)\l && \<=. x1.rain (+. -10 x2.rain))\l && \>=. x1.temp (+. 10 x2.temp))\l \|\| \>=.\l (if \<=.\l (+.\l (+.\l (+.\l (+.\l (+.\l (*. -20407311067/10000000000\l (if \<=.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+.\l (+. (*. 3717/100000 (Real.of_int x1.x))\l (*. -821/175000 (Real.of_int x1.y)))\l (*. 8441/50000\l (if x1.month = Jan then 67/500\l else\l if x1.month = Feb then 1/2\l else\l if x1.month = Mar then 1\l else if x1.month = Apr then 3/2 else …)))\l (*. 15503/95050\l (if x1.day = Mon then 377/1000\l else\l if x1.day = Tue then 1223/1000\l else if x1.day = Wed then 1901/1000 else …)))\l …)\l …)\l …)\l …)\l …)\l …)\l …)\l …)\l 23704241183297452571/127754212462947000000\l then 0 else …))\l …)\l …)\l …)\l …)\l …)\l -4742192817/10000000000\l then … else …)\l …)",shape=box,style=filled,color="cyan",fontname="courier",fontsize=14];
}
 Load graph Loading.. proof attempt summary full graph ground_instances 0 definitions 0 inductions 0 search_time 59.553s Expand start[59.553s]
 (0 <= :var_0:.x
 && :var_0:.x <= 9
 && 0 <= :var_0:.y
 && :var_0:.y <= 9
 && <=. 0 :var_0:.ffmc
 && <=. :var_0:.ffmc 99
 && <=. 0 :var_0:.dmc
 && <=. :var_0:.dmc 500
 && <=. 0 :var_0:.dc
 && <=. :var_0:.dc 1000
 && <=. 0 :var_0:.isi
 && <=. :var_0:.isi 100
 && <=. 0 :var_0:.temp
 && <=. :var_0:.temp 40
 && <=. 0 :var_0:.rh
 && <=. :var_0:.rh 100
 && <=. 0 :var_0:.wind
 && <=. :var_0:.wind 15
 && <=. 0 :var_0:.rain
 && <=. :var_0:.rain
 15)
 && (0 <= :var_1:.x
 && :var_1:.x <= 9
 && 0 <= :var_1:.y
 && :var_1:.y <= 9
 && <=. 0 :var_1:.ffmc
 && <=. :var_1:.ffmc 99
 && <=. 0 :var_1:.dmc
 && <=. :var_1:.dmc 500
 && <=. 0 :var_1:.dc
 && <=. :var_1:.dc 1000
 && <=. 0 :var_1:.isi
 && <=. :var_1:.isi 100
 && <=. 0 :var_1:.temp
 && <=. :var_1:.temp 40
 && <=. 0 :var_1:.rh
 && <=. :var_1:.rh 100
 && <=. 0 :var_1:.wind
 && <=. :var_1:.wind 15
 && <=. 0
 :var_1:.rain
 && <=.
 :var_1:.rain
 15)
 && :var_0:.x = :var_1:.x
 && :var_0:.y = :var_1:.y
 && :var_0:.month = :var_1:.month
 && not
 (:var_0:.month = Oct
 || :var_0:.month = Nov
 || :var_0:.month = Dec
 || :var_0:.month = Jan || :var_0:.month = Feb)
 && :var_0:.day = :var_1:.day
 && :var_0:.ffmc = :var_1:.ffmc
 && :var_0:.dmc = :var_1:.dmc
 && :var_0:.dc = :var_1:.dc
 && :var_0:.isi = :var_1:.isi
 && :var_0:.rh = :var_1:.rh
 && :var_0:.wind = :var_1:.wind
 && <=. (+. :var_0:.rain 10)
 :var_1:.rain
 && >=. (-. :var_0:.temp 10)
 :var_1:.temp
 ==> >=.
 (+.
 (if <=.
 (*. 444323/100000
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (*. -45929/100000
 (if >.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+.
 (+. (*. 3717/12500 …) (*. -821/25000 …))
 (*. 8441/25000 …))
 (*. 15503/50000 …))
 (*. 3883/

It should be noted of course that Imandra doesn't know the likelihood of any of the instances or counterexamples it generates. Thus in practice it may be deemed that any counterexamples are sufficiently unlikely that the model is still accurate and safe enough to use. A further safety property we might to wish to prove of our model is robustness; namely that similar inputs should generate similar outputs. Proving this for the whole model in one go is computationally very difficult, but we can instead prove a robustness property for each layer and then chain them together for our final proof.

In [ ]:
let abs_dist_input a b =
let (a_0, a_1, a_2, a_3, a_4, a_5, a_6, a_7, a_8, a_9, a_10, a_11) = process_input a in
let (b_0, b_1, b_2, b_3, b_4, b_5, b_6, b_7, b_8, b_9, b_10, b_11) = process_input b in
Real.abs(a_0 -. b_0) +.
Real.abs(a_1 -. b_1) +.
Real.abs(a_2 -. b_2) +.
Real.abs(a_3 -. b_3) +.
Real.abs(a_4 -. b_4) +.
Real.abs(a_5 -. b_5) +.
Real.abs(a_6 -. b_6) +.
Real.abs(a_7 -. b_7) +.
Real.abs(a_8 -. b_8) +.
Real.abs(a_9 -. b_9) +.
Real.abs(a_10 -. b_10) +.
Real.abs(a_11 -. b_11);;

let abs_dist_hidden a b =
let (a_0, a_1, a_2, a_3, a_4, a_5) = a in
let (b_0, b_1, b_2, b_3, b_4, b_5) = b in
Real.abs(a_0 -. b_0) +.
Real.abs(a_1 -. b_1) +.
Real.abs(a_2 -. b_2) +.
Real.abs(a_3 -. b_3) +.
Real.abs(a_4 -. b_4) +.
Real.abs(a_5 -. b_5);;

let abs_dist_output a b =
let a_0 = a in
let b_0 = b in
Real.abs(a_0 -. b_0);;

(* verify (fun a b -> is_valid a && is_valid b ==> (abs_dist_input (process_input a) (process_input b)) <=. 1.0);;
 *)
 
lemma input_hidden a b = abs_dist_input a b <=. 0.5 ==> 
                         abs_dist_hidden (a |> process_input |> layer_0) (b |> process_input |> layer_0) <=. 6.0 [@@auto];;
                         
(* lemma hidden_output a b = abs_dist_hidden a b <=. 6.0 ==> 
                             abs_dist_output (a |> layer_1) (b |> layer_1) <=. 20.0;; *)

(* verify (fun a b -> (is_valid a && is_valid b && abs_dist_input a b <=. 0.5) ==> Real.abs(model a -. model b) <=. 100.0)
 *)


In [13]:
(* Decompose.top ~assuming:"is_valid" "model";; *)

In [67]:
model CX.a;;
model CX.b;;

- : real = 1.71828
- : real = 101.75783726


In [2]:
#config

----------------------------------------------------------------------------
Imandra session configuration
----------------------------------------------------------------------------
 0. Timeout: 60000
 1. Mode: Logic
 2. Allow redefinition: true
 3. Load path: []
 4. Recursion unroll depth: 100
 5. Random testing: false
 6. SAT seed: 0
 7. SMT seed: 0
 8. Solver seed: 0
 9. Eager goal-based GC: false
10. Undo (implicit push): false
11. Console tags: waterfall, suggestions
12. Console color: true
13. Top results: true
14. Interactive hints: true
15. Validate definitions: true
16. Skip proofs: false
17. Max induction depth: 3
18. Induction unroll depth: 10
19. Backchain limit: 100
20. Enable all definitions when unrolling: true
21. Algebraic number approx. precision: 10
22. Reflect approximate models: false
----------------------------------------------------------------------------
